In [2]:
import pandas as pd
import matplotlib as plt
import rasterio
import numpy as np

In [ ]:
df = pd.read_csv("C:/Users/User/PycharmProjects/SafeRoute/NYPD_Arrests_Data__Historic_.csv")
df = df.replace('(null)', np.nan)
df = df.replace('nan', np.nan)
df = df.dropna(axis=0)
df

In [ ]:
# np.unique(np.array())
list(set(df['OFNS_DESC']))

In [ ]:
danger_scores = {
    'HOMICIDE-NEGLIGENT,UNCLASSIFIED': 10,
    'SEX CRIMES': 9,
    'GRAND LARCENY': 3,
    'ROBBERY': 8,
    'UNLAWFUL POSS. WEAP. ON SCHOOL GROUNDS': 6,
    'KIDNAPPING & RELATED OFFENSES': 9,
    'BURGLARY': 6,
    'DANGEROUS DRUGS': 5,
    'FRAUDS': 2,
    'RAPE': 12,
    'FELONY SEX CRIMES': 10,
    'ASSAULT 3 & RELATED OFFENSES': 8,
    'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)': 4,
    'FELONY ASSAULT': 9,
    'MURDER & NON-NEGL. MANSLAUGHTER': 15,
    'THEFT-FRAUD': 3,
    'PETIT LARCENY': 2
}

In [ ]:
new_df = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), ['ARREST_DATE', 'Latitude', 'Longitude']]
new_df

In [ ]:
new_df['Danger Score'] = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), 'OFNS_DESC'].map(danger_scores)
new_df

In [ ]:
new_df.sort_values(by=['Longitude'])

In [ ]:
(new_df['Latitude'].min(), new_df['Latitude'].max(), new_df['Longitude'].min(), new_df['Longitude'].max())

In [ ]:
with rasterio.open("C:/Users/User/PycharmProjects/SafeRoute/clustered.tif") as src:
    data = src.read()
    image = src.read(1)
    clusters = src.read(2)
    bounds = src.bounds

In [ ]:
bounds

In [ ]:
(abs(bounds[2] - bounds[0]), abs(bounds[3] - bounds[1]))

In [ ]:
step = (abs(bounds[2] - bounds[0])/np.shape(clusters)[1] + abs(bounds[3] - bounds[1])/np.shape(clusters)[0]) / 2
step

In [ ]:
cluster_score = {c: 0 for c in np.unique(clusters)}
cluster_crime_num = {c: 0 for c in np.unique(clusters)}
cluster_score

In [ ]:
for row in new_df.itertuples():
    p = (row.Longitude, row.Latitude) # point declaration
    if abs(p[0])>abs(bounds[0]) or abs(p[0])<abs(bounds[2]): 
        continue
    elif abs(p[1])<abs(bounds[1]) or abs(p[1])>abs(bounds[3]): 
        continue 
    cluster_score[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += row._4
    cluster_crime_num[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += 1

In [ ]:
cluster_score

In [ ]:
cluster_score[-1] = 0
cluster_crime_num[-1] = 0

HOUSE PRICES DATABASE